# Multi-hospital Organ Transplants

This notebook explores expectations related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario, and **Expectations.ipynb** for a discussion of expectations in this context. Both of these notebooks are located in the parent repository, **MultipleAgentsEC**.

### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

```diff
- IMPORTANT NOTE: you should run AgentsAndEvents.ipynb prior to running this notebook to ensure that AE.pl contains up to date predicates! You must run the notebook, not just update and save it.
```

In [1]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

# Authorisation

An agent may expect that other agents that it interacts with will be authorised in some manner. Here, we declare that **monSys** expects that waiting list add requests and donation offers come from agents that **monSys** views to be registered hospitals, which are hospitals that were initially known to **monSys**.

We also declare that **monSys** expects that the transplant location specified in a match event is a hospital declared to be known by **monSys**.

In [2]:
% File: expAuthorisation.pl

% Test expectation rule - hospital wait list requests should come from recognised agent.
initially(monSys:exp_rule(happ(receive(Hospital, waitAddReq(_, _, _))), agent(Hospital), dependent, 
"Authorisation":"Wait list requests should come from recognised hospital")).
initially(monSys:exp_rule(happ(receive(Hospital, donorOffer(PatID, Organs, Details))), agent(Hospital),
dependent, "Authorisation":"Donation offers should come from a recognised hospital")).

initially(monSys:exp_rule(happ(match(_, _, _, _, _, Location, _, _)), agent(Location), dependent, 
"Authorisation":"Transplant location should be a registered hospital")).

In [3]:
% File: narrative.pl

happensAtNarrative("Otago":send(monSys, waitAddReq(104, ["heart"], [bloodType:"A"])),0).
happensAtNarrative("Nelson":send(monSys, waitAddReq(105, ["liver"], [bloodType:"AB", dob:(4,11,1950)])),4).


happensAtNarrative("Hamilton": send(monSys, donorOffer(200, [lungs, pancreas], [bloodType:"O"])),6).

happensAtNarrative("Wellington": send(monSys, donorOffer(201, [heart, kidney, liver, lungs, pancreas], [bloodType:"A"])),8).

happensAtNarrative(monSys:match(701, "Wellington", 201, "Otago", 104, "Timaru", ["heart"], []),10).
happensAtNarrative(monSys:match(701, "Wellington", 201, "Otago", 104, "Auckland", ["heart"], []),13).

We set up our environment for 20 time periods, so that we can query events and fluents in this time frame.

In [4]:
?- run(20).

true.

The hospitals that **monSys** recognises are Otago, Christchurch, Wellington and Auckland.

In [5]:
?- initially(monSys:agent(Hospital)).

Hospital = b'Otago' ;
Hospital = b'Christchurch' ;
Hospital = b'Wellington' ;
Hospital = b'Auckland' .

At time period 1, **monSys** receives the waiting list add request sent by Otago. The expectation that a request will come from a recognised hospital is fulfilled.

In [6]:
?- happensAt(Hosp, Event, 1).


Hosp = monSys, Event = fulf(happ(receive(b'Otago', waitAddReq(104, [b'heart'], [Functor(188685,2,bloodType,b'A')]))), agent(b'Otago'), 1, agent(b'Otago'), dependent, :(b'Authorisation', b'Wait list requests should come from recognised hospital')) ;
Hosp = monSys, Event = receive(b'Otago', waitAddReq(104, [b'heart'], [Functor(188685,2,bloodType,b'A')])) .

As there were no expectation violations caused by the waiting list add request, **monSys** automatically accepts the request and notifies Otago.

In [7]:
?- T = 2, happensAt(Hosp, Event, T).
?- T = 3, happensAt(Hosp, Event, T).

T = 2, Hosp = monSys, Event = send(b'Otago', waitAccept(104)) .
T = 3, Hosp = b'Otago', Event = receive(monSys, waitAccept(104)) .

At time period 5, **monSys** receives a waiting list add request from Nelson hospital. As Nelson is not a recognised hospital, this causes a violation of **monSys**' expectation that waiting list add requests will come from registered hospitals.

In [8]:
?- happensAt(monSys, Event, 5).

Event = viol(happ(receive(b'Nelson', waitAddReq(105, [b'liver'], [Functor(188685,2,bloodType,b'AB'), Functor(188685,2,dob,,(4, ,(11, 1950)))]))), agent(b'Nelson'), 5, agent(b'Nelson'), dependent, :(b'Authorisation', b'Wait list requests should come from recognised hospital')) ;
Event = receive(b'Nelson', waitAddReq(105, [b'liver'], [Functor(188685,2,bloodType,b'AB'), Functor(188685,2,dob,,(4, ,(11, 1950)))])) .

Due to the violation, **monSys** sends a waiting list request rejection message to Nelson, telling Nelson that they are not authorised to perform such an action.

In [9]:
?- happensAt(monSys, Event, 6).

Event = send(b'Nelson', waitReject(105, :(b'Authorisation', b'Wait list requests should come from recognised hospital'))) .

At time period 7, **monSys** receives a donor offer from Hamilton hospital. This creates an authorisation violation.

In [10]:
?- happensAt(monSys, Event, 7).

Event = viol(happ(receive(b'Hamilton', donorOffer(200, [Atom('672773'), Atom('672901')], [Functor(188685,2,bloodType,b'O')]))), agent(b'Hamilton'), 7, agent(b'Hamilton'), dependent, :(b'Authorisation', b'Donation offers should come from a recognised hospital')) ;
Event = receive(b'Hamilton', donorOffer(200, [Atom('672773'), Atom('672901')], [Functor(188685,2,bloodType,b'O')])) .

At time period 9, **monSys** receives a donor offer from Wellington hospital, which creates an authorisation fulfilment.

In [12]:
?- happensAt(monSys, Event, 9).

Event = fulf(happ(receive(b'Wellington', donorOffer(201, [Atom('673029'), Atom('673157'), Atom('673285'), Atom('672773'), Atom('672901')], [Functor(188685,2,bloodType,b'A')]))), agent(b'Wellington'), 9, agent(b'Wellington'), dependent, :(b'Authorisation', b'Donation offers should come from a recognised hospital')) ;
Event = receive(b'Wellington', donorOffer(201, [Atom('673029'), Atom('673157'), Atom('673285'), Atom('672773'), Atom('672901')], [Functor(188685,2,bloodType,b'A')])) .

At time period 10, **monSys** matches a donor and recipient, and specifies the transplant location as Timaru. This creates an authorisation violation.

In [14]:
?- happensAt(monSys, Event, 10).

Event = viol(happ(match(701, b'Wellington', 201, b'Otago', 104, b'Timaru', [b'heart'], [])), agent(b'Timaru'), 10, agent(b'Timaru'), dependent, :(b'Authorisation', b'Transplant location should be a registered hospital')) ;
Event = match(701, b'Wellington', 201, b'Otago', 104, b'Timaru', [b'heart'], []) .

At time period 10, **monSys** matches a donor and recipient. All hospitals specified in the match are registered, so this event creates an authorisation violation.

In [15]:
?- happensAt(monSys, Event, 13).

Event = fulf(happ(match(701, b'Wellington', 201, b'Otago', 104, b'Auckland', [b'heart'], [])), agent(b'Auckland'), 13, agent(b'Auckland'), dependent, :(b'Authorisation', b'Transplant location should be a registered hospital')) ;
Event = match(701, b'Wellington', 201, b'Otago', 104, b'Auckland', [b'heart'], []) .

We can query all the violations which occur in our loaded time frame (25 time periods).

In [16]:
?- happensAt(Hospital, viol(_,_,_,_,_,Message), T).

Hospital = monSys, Message = :(b'Authorisation', b'Wait list requests should come from recognised hospital'), T = 5 ;
Hospital = monSys, Message = :(b'Authorisation', b'Donation offers should come from a recognised hospital'), T = 7 ;
Hospital = monSys, Message = :(b'Authorisation', b'Transplant location should be a registered hospital'), T = 10 .

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>